# How to Use:
This Notebook should ease the use of the API for "cleanup" tasks. It includes some sample for specific use cases that can be adjusted.
1. run first block to prepare excecutions by creating api connection
2. run any following block to execute a specific function

In [ ]:
import json
import logging
import logging.config
from pathlib import Path

from churchtools_api.churchtools_api import ChurchToolsApi
from secure.config import ct_domain, ct_token

logger = logging.getLogger(__name__)

config_file = Path("logging_config.json")
with config_file.open(encoding="utf-8") as f_in:
    logging_config = json.load(f_in)
    log_directory = Path(logging_config["handlers"]["file"]["filename"]).parent
    if not log_directory.exists():
        log_directory.mkdir(parents=True)
    logging.config.dictConfig(config=logging_config)

# Create Session


api = ChurchToolsApi(ct_domain)
api.login_ct_rest_api(ct_token=ct_token)

## Songs & Tag - mass change
Helper to iterate all songs

In [ ]:
songs = api.get_songs()
print('Got {} songs and the first one is "{}"'.format(len(songs), songs[0]["name"]))  # noqa: T201

Mass change append a tag to all songs - e.g. 51 is tag:"in ChurchTools vor Skript Import"

In [ ]:
all_song_ids = [value["id"] for value in songs]
for song_id in all_song_ids:
    api.add_song_tag(song_id=song_id, song_tag_id=51)

# Mass import Gartenschau Termine

## prepare dataset

### from TXT

In [ ]:
import re
from pathlib import Path

import pandas as pd

pd.set_option("display.max_colwidth", None)

with Path("import/gartenschau.txt").open() as file:
    raw = file.readlines()

PATTERN = r"([\d.]+) +(\d\d:\d\d) +(.+)"


lines = []
for line in raw:
    line.replace("  ", " ")
    data = line[:-1].split(";")
    split = re.match(pattern=PATTERN, string=data[1].strip()).groups()
    lines.append([data[0].strip(), *split, *data[2:]])

df_imported = pd.DataFrame(lines)
df_imported

In [ ]:
df_imported["start_date"] = pd.to_datetime(
    df_imported[1] + "2025T" + df_imported[2], format="%d.%m.%YT%H:%M"
)
df_imported["title"] = df_imported[3]
df_imported["subtitle"] = df_imported[4]
df_imported["notes"] = df_imported[5]

### Cleanup using spreadsheet

In [ ]:
df_imported = df_imported[["start_date", "title", "subtitle", "notes"]]
df_imported["title"] = df_imported["title"].str.replace("Ökum. ", "")
df_imported = df_imported.fillna("")

Correcting with spreadsheed summary

In [ ]:
# df_imported.to_excel("import/gartenschau.xlsx")  # noqa: ERA001
df_new = pd.read_excel("import/gartenschau.xlsx")
df_new = df_new.fillna("")
df_imported["title"] = df_new.apply(
    lambda row: row["title"] + " (" + row["title2"] + ")"
    if row["title2"]
    else row["title"],
    axis=1,
)
disclaimer = " - Die Gottesdienste der Gartenschau werden von den Kirchen der ACK Freudenstadt und Baiersbronn gemeinsam veranstaltet. Die Organisation des jeweiligen Termins obliegt der jeweiligen Gruppe, kurzfristige Änderungen vorbehalten."
df_imported["notes"] = df_imported.apply(lambda row: row["notes"] + disclaimer, axis=1)
df_imported

## prepare common data for calendar appointment

In [ ]:
from datetime import timedelta

import pytz

address = {
    "meetingAt": "Volksbank Bühne",
    "city": "Baiersbronn",
    "street": "Am Königshammer",
    "country": "DE",
    "district": "Friedrichstal",
    "latitude": "48.4824546",
    "longitude": "8.3756814",
    "zip": "72270",
}

GARTENSCHAU_GODI = 73
GARTENSCHAU_KONZERT = 76
GARTENSCHAU_SONSTIGE = 82
GARTENSCHAU_KIDNER = 79

cest = pytz.timezone("Europe/Berlin")

create events

## Delete all existing Events

In [ ]:
from datetime import datetime

events = api.get_calendar_appointments(
    calendar_ids=[GARTENSCHAU_GODI],
    from_=datetime(year=2025, month=5, day=1),
    to_=datetime(year=2025, month=10, day=30),
)
for item in events:
    api.delete_calender_appointment(
        calendar_id=GARTENSCHAU_GODI, appointment_id=item["id"]
    )

## Create all new entries

In [ ]:
created_appointment_ids = []

for _index, line in df_imported.iterrows():
    new_id = api.create_calender_appointment(
        calendar_id=GARTENSCHAU_GODI,
        startDate=cest.localize(line["start_date"]),
        endDate=cest.localize(line["start_date"]) + timedelta(minutes=75),
        title=line["title"],
        subtitle=line["subtitle"],
        description=line["notes"],
        address=address,
    )
    created_appointment_ids.append(new_id)

created_ids = [item["id"] for item in created_appointment_ids]